In [1]:
from Bio import PDB
from Bio.PDB import Superimposer
from Bio.PDB import DSSP
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
from pathlib import Path
import os
import csv
import pandas as pd
from scipy.spatial import distance
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
def process_directory(directory):
    for dirpath, dirname, filenames in os.walk(directory):
        batch_number = os.path.basename(dirpath)
        for filename in filenames:
            if filename.endswith(".cif"):
                continue

In [3]:
def calculate_rmsd(fixed_atoms, moving_atoms, structure):
    super_imposer = Superimposer()
    super_imposer.set_atoms(fixed_atoms, moving_atoms)
    super_imposer.apply(structure.get_atoms())
    return super_imposer.rms

In [3]:
output_file = '/home/max/stayahead/snellius/outputs/alpha_var.txt'

In [4]:
p = PDB.PDBParser(QUIET=True)
struct_path = Path('/home/max/stayahead/snellius2/outputs/references/alphafold/SARS-CoV-ACE2-AF_1.pdb')
structure = p.get_structure('fixed', struct_path)
parent_dir = struct_path.parent
key_name = parent_dir.name
print(key_name)
crys_struct = p.get_structure('crystal', '/home/max/stayahead/snellius2/outputs/references/alphafold/SARS-CoV-ACE2_2/ranked_0.pdb')
crys_model = crys_struct[0]
for chain in crys_model:
    if chain.id == 'E':
        fixed_crys_atoms = [residue['CA'] for residue in chain.get_residues() if 'CA' in residue]
        moving_ref_atoms = [residue['CA'] for residue in structure.get_residues() if 'CA' in residue]
        print(len(fixed_crys_atoms))
        print(len(moving_ref_atoms))
        dif = len(fixed_crys_atoms) - len(moving_ref_atoms)
        print(dif)
                # Adjust the length of moving_ref_atoms based on dif
        if dif < 0:
            # If dif is negative, remove the last -dif atoms from moving_ref_atoms
            moving_ref_atoms = moving_ref_atoms[:dif]
        elif dif > 0:
            # If dif is positive, you might need to address how to handle this case,
            # such as choosing which atoms to remove from fixed_crys_atoms or reconsidering alignment.
            pass
        # moving_ref_atoms.pop()
        assert len(fixed_crys_atoms) == len(moving_ref_atoms)
        rmsd = calculate_rmsd(fixed_crys_atoms, moving_ref_atoms, chain)
        # with open(output_file, 'w') as f:
        #     f.write(f'{rmsd}\n')
        print(rmsd)


alphafold


In [5]:
p = PDB.PDBParser(QUIET=True)
struct_path = Path('/home/max/stayahead/snellius2/outputs/references/alphafold/SARS-CoV-ACE2-AF_1.pdb')
structure = p.get_structure('moving', struct_path)
parent_dir = struct_path.parent
key_name = parent_dir.name
print(key_name)
# ref_struct = p.get_structure('fixed', '/home/max/stayahead/snellius2/outputs/af/ranked_0.pdb')
ref_struct = p.get_structure('fixed', '/home/max/stayahead/snellius2/outputs/references/alphafold/SARS-CoV-ACE2_2/ranked_0.pdb')
moving_ref_atoms = [residue['CA'] for residue in structure.get_residues() if 'CA' in residue]
fixed_ref_atoms = [residue['CA'] for residue in ref_struct.get_residues() if 'CA' in residue]
print(len(fixed_ref_atoms))
print(len(moving_ref_atoms))
dif = len(fixed_ref_atoms) - len(moving_ref_atoms)
print(dif)
        # Adjust the length of moving_ref_atoms based on dif
if dif < 0:
    # If dif is negative, remove the last -dif atoms from moving_ref_atoms
    moving_ref_atoms = moving_ref_atoms[:dif]
elif dif > 0:
    # If dif is positive, you might need to address how to handle this case,
    # such as choosing which atoms to remove from fixed_crys_atoms or reconsidering alignment.
    pass
# moving_ref_atoms.pop()
assert len(fixed_ref_atoms) == len(moving_ref_atoms)
rmsd = calculate_rmsd(fixed_ref_atoms, moving_ref_atoms, ref_struct)
print(rmsd)

alphafold
195
195
0
0.1785627251075748


In [4]:
print(len('RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFHHHHHH'))

229


In [6]:
import freesasa
structure = freesasa.Structure('/home/max/stayahead/snellius/outputs/ds4/alpha_variant/ranked_0.pdb')
result = freesasa.calc(structure)
total_sasa = result.totalArea()
# dict[self.n] = total_sasa
print(total_sasa)

10332.772835983595


In [7]:
asdf = str(struct_path)

In [16]:
from Bio import PDB
from Bio.PDB import Superimposer
from Bio.PDB import DSSP
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
from pathlib import Path
import os
import csv
import argparse
# from scipy.spatial import distance
import numpy as np
# import matplotlib.pyplot as plt
import math
from tmtools.io import get_structure, get_residue_data
from tmtools import tm_align
import json
import freesasa

class ComputeMetrics():
    def __init__(self, args):
        self.args = args
        self.input = args.input
        self.output = args.out_dir
        self.ref = args.ref
        # q = query, r = reference, n = name
        self.q, self.r, self.n = self.load_structures()

    
    def load_structures(self):
        p = PDB.PDBParser(QUIET=True)
        struct_path = Path(self.input)
        ref_path = Path(self.ref)
        parent_dir = struct_path.parent
        key_name = parent_dir.name
        fixed_struct = p.get_structure('fixed', ref_path)
        moving_struct = p.get_structure('moving', struct_path)

        return fixed_struct, moving_struct, key_name

    def calculate_rmsd(fixed_atoms, moving_atoms, structure):
        super_imposer = Superimposer()
        super_imposer.set_atoms(fixed_atoms, moving_atoms)
        super_imposer.apply(structure.get_atoms())
        return super_imposer.rms
    
    def rmsd(self, dict):
        fixed_model = self.r[0]
        for chain in fixed_model:
            if chain.id == 'E':
                all_atoms_fixed = [atom for residue in chain.get_residues() for atom in residue]
                all_atoms_moving = [atom for residue in self.q.get_residues() for atom in residue]
                if len(all_atoms_fixed) > len(all_atoms_moving):
                    dif = len(all_atoms_fixed) - len(all_atoms_moving)
                    all_atoms_fixed = all_atoms_fixed[:len(all_atoms_fixed) - dif]
                else:
                    dif = len(all_atoms_moving) - len(all_atoms_fixed)
                    all_atoms_moving = all_atoms_moving[:len(all_atoms_moving) - dif]
                    # If dif is positive, you might need to address how to handle this case,
                    # such as choosing which atoms to remove from fixed_crys_atoms or reconsidering alignment.
                assert len(all_atoms_fixed) == len(all_atoms_moving)
                rmsd = self.calculate_rmsd(all_atoms_fixed, all_atoms_moving, chain)
                dict[self.n] = rmsd

        return dict

    def tm_score(self, dict):
        fixed_model = self.r[0]
        for chain in fixed_model:
            if chain.id == 'E':
                all_atoms_fixed = [atom for residue in chain.get_residues() for atom in residue]
                all_atoms_moving = [atom for residue in self.q.get_residues() for atom in residue]
                if len(all_atoms_fixed) > len(all_atoms_moving):
                    dif = len(all_atoms_fixed) - len(all_atoms_moving)
                    all_atoms_fixed = all_atoms_fixed[:len(all_atoms_fixed) - dif]
                else:
                    dif = len(all_atoms_moving) - len(all_atoms_fixed)
                    all_atoms_moving = all_atoms_moving[:len(all_atoms_moving) - dif]
                    # If dif is positive, you might need to address how to handle this case,
                    # such as choosing which atoms to remove from fixed_crys_atoms or reconsidering alignment.
                assert len(all_atoms_fixed) == len(all_atoms_moving)
                tm_score = self.calculate_tm_score(all_atoms_fixed, all_atoms_moving, chain)
                dict[self.n] = tm_score

        return dict
    
    def sasa(self, dict):
        structure = freesasa.Structure(str(self.input))
        result = freesasa.calc(structure)
        total_sasa = result.totalArea()
        dict[self.n] = total_sasa
        # res_dict = {}
        # for i, residue in enumerate(structure.residueAreas()):
        #     residue_sasa = sum(residue.total)
        #     res_dict[i] = residue_sasa
        return dict

In [12]:
def create_parser():
    parser = argparse.ArgumentParser(description="Compute structural metrics.")
    parser.add_argument(
        "-i", "--input", help="Path to input sequence", type=Path, required=True,
    )
    parser.add_argument(
        "-o", "--out_dir", help="Path to save directory for sequences", type=Path, required=True,
    )
    parser.add_argument(
        "-r", "--ref", help="Path to reference structure", type=Path, required=True,
    )
    return parser

parser = create_parser()
args = parser.parse_args('-i /home/max/stayahead/snellius/outputs/ds4/alpha_variant/ranked_0.pdb -o /home/max/stayahead/snellius/outputs/ds4/alpha_variant -r /home/max/stayahead/esm/input/6m0j.pdb'.split())


In [13]:
metrics = ComputeMetrics(args)

In [17]:
rmsd_dict = {}
tm_score_dict = {}
sasa_dict = {}
sasa_dict = metrics.sasa(sasa_dict)
rmsd_dict = metrics.rmsd(rmsd_dict)
tm_score_dict = metrics.tm_score(tm_score_dict)

In [18]:
print(rmsd_dict)

{}


In [2]:
from Bio.PDB import PDBParser, DSSP

# Load your structure file
parser = PDBParser()
structure = parser.get_structure("protein", "/home/max/stayahead/snellius2/outputs/ACE2.pdb")

# Perform DSSP analysis
model = structure[0]
dssp = DSSP(model, "/home/max/stayahead/snellius2/outputs/ACE2.pdb")

# Extract secondary structure information
sec_struct_counts = {"H": 0, "B": 0, "E": 0, "G": 0, "I": 0, "T": 0, "S": 0, "-": 0}
for key in dssp.keys():
    ss = dssp[key][2]
    sec_struct_counts[ss] += 1

total_residues = sum(sec_struct_counts.values())
sec_struct_proportions = {k: v / total_residues for k, v in sec_struct_counts.items()}
print(sec_struct_proportions)

{'H': 0.1134020618556701, 'B': 0.030927835051546393, 'E': 0.25257731958762886, 'G': 0.061855670103092786, 'I': 0.0, 'T': 0.0979381443298969, 'S': 0.12371134020618557, '-': 0.31958762886597936}


In [5]:
from Bio.PDB import PDBParser, Polypeptide
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.SeqUtils.ProtParamData import kd

# Function to extract sequence from PDB file
def extract_sequence_from_pdb(pdb_file):
    parser = PDBParser()
    structure = parser.get_structure("protein", pdb_file)
    model = structure[0]
    ppb = Polypeptide.PPBuilder()
    for pp in ppb.build_peptides(model):
        return str(pp.get_sequence())

# Function to calculate hydrophobicity
def calculate_hydrophobicity(sequence):
    protein_analysis = ProteinAnalysis(sequence)
    hydrophobicity = protein_analysis.protein_scale(kd, window=7, edge=1.0)
    return hydrophobicity

def calculate_total_hydrophobicity(sequence):
    total_hydrophobicity = 0
    for aa in sequence:
        if aa in kd:
            total_hydrophobicity += kd[aa]
    return total_hydrophobicity / len(sequence)



# Path to PDB file
pdb_file = "/home/max/stayahead/snellius2/outputs/ACE2.pdb"

# Extract sequence and calculate hydrophobicity
sequence = extract_sequence_from_pdb(pdb_file)
hydrophobicity = calculate_hydrophobicity(sequence)
average_hydrophobicity = sum(hydrophobicity) / len(hydrophobicity)
total_hydrophobicity = calculate_total_hydrophobicity(sequence)

print(f"Average Hydrophobicity: {average_hydrophobicity}")
print(f"Total Hydrophobicity: {total_hydrophobicity}")
print(f"Protein sequence: {sequence}")
print(f"Hydrophobicity: {hydrophobicity}")

Average Hydrophobicity: -0.2442249240121583
Total Hydrophobicity: -0.21649484536082475
Protein sequence: TNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCG
Hydrophobicity: [0.4142857142857142, 0.014285714285714235, 1.1142857142857143, 0.9714285714285714, 0.1142857142857144, 0.5999999999999999, 0.10000000000000002, -0.48571428571428577, 0.4142857142857142, 0.07142857142857133, -0.4428571428571429, 0.6571428571428571, 0.21428571428571427, 0.5714285714285714, 1.0857142857142856, 0.18571428571428567, -0.7142857142857143, -1.157142857142857, -2.4, -1.5714285714285714, -1.9428571428571428, -2.3142857142857145, -1.457142857142857, -0.21428571428571425, 0.5999999999999999, 0.7428571428571429, -0.08571428571428566, -0.08571428571428563, 1.0142857142857145, 1.1999999999999997, 0.4142857142857143, -0.3428571428571428, 0.04285714285714283, 0.48571428571428

In [5]:
from Bio import PDB
from Bio.PDB import Superimposer
from Bio.PDB import DSSP
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
from pathlib import Path
import os
import csv
import argparse
# from scipy.spatial import distance
import numpy as np
# import matplotlib.pyplot as plt
import math
from tmtools.io import get_structure, get_residue_data
from tmtools import tm_align
import json
import freesasa
from Bio.PDB import PDBParser, Polypeptide
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.SeqUtils.ProtParamData import kd
from Bio.SeqUtils.ProtParamData import hw

class ComputeMetrics():
    def __init__(self, input_path, ref_path):
        self.input = input_path
        self.ref = ref_path
        # q = query, r = reference, n = name
        # self.q, self.r, self.n = self.load_structures()

    
    def load_structures(self):
        p = PDB.PDBParser(QUIET=True)
        struct_path = Path(self.input)
        ref_path = Path(self.ref)
        parent_dir = struct_path.parent
        key_name = parent_dir.name
        fixed_struct = p.get_structure('fixed', ref_path)
        moving_struct = p.get_structure('moving', struct_path)

        return fixed_struct, moving_struct, key_name

    def calculate_rmsd(self, fixed_atoms, moving_atoms, structure):
        super_imposer = Superimposer()
        super_imposer.set_atoms(fixed_atoms, moving_atoms)
        super_imposer.apply(structure.get_atoms())
        return super_imposer.rms
    
    def rmsd(self):
        # fixed_model = self.r[0]
        # p = PDB.PDBParser(QUIET=True)
        # struct_path = Path(self.input)
        # ref_path = Path(self.ref)
        # parent_dir = struct_path.parent
        # key_name = str(parent_dir.name)
        # fixed_struct = p.get_structure('fixed', ref_path)
        # moving_struct = p.get_structure('moving', struct_path)
        fixed_struct, moving_struct, key_name = self.load_structures()
        fixed_model = fixed_struct[0]
        for chain in fixed_model:
            all_atoms_fixed = [atom for residue in chain.get_residues() for atom in residue]
            all_atoms_moving = [atom for residue in moving_struct.get_residues() for atom in residue]
            if len(all_atoms_fixed) > len(all_atoms_moving):
                dif = len(all_atoms_fixed) - len(all_atoms_moving)
                all_atoms_fixed = all_atoms_fixed[:len(all_atoms_fixed) - dif]
            else:
                dif = len(all_atoms_moving) - len(all_atoms_fixed)
                all_atoms_moving = all_atoms_moving[:len(all_atoms_moving) - dif]
                # If dif is positive, you might need to address how to handle this case,
                # such as choosing which atoms to remove from fixed_crys_atoms or reconsidering alignment.
            # print(len(all_atoms_fixed), len(all_atoms_moving))
            assert len(all_atoms_fixed) == len(all_atoms_moving)
            rmsd = self.calculate_rmsd(all_atoms_fixed, all_atoms_moving, chain)

                    # print(rmsd)
        return rmsd
    

    def tm_score(self):
        ref_struct, query_struct, key_name = self.load_structures()
        chain = next(ref_struct.get_chains())
        ref_coords, ref_seq = get_residue_data(chain)
                    # If dif is positive, you might need to address how to handle this case,
                    # such as choosing which atoms to remove from fixed_crys_atoms or reconsidering alignment.
        chain = next(query_struct.get_chains())
        coords, seq = get_residue_data(chain)
        res = tm_align(ref_coords, coords, ref_seq, seq)
        tm_score = res.tm_norm_chain1

        return tm_score
    
    def sasa(self):
        structure = freesasa.Structure(str(self.input))
        result = freesasa.calc(structure)
        total_sasa = result.totalArea()
        # res_dict = {}
        # for i, residue in enumerate(structure.residueAreas()):
        #     residue_sasa = sum(residue.total)
        #     res_dict[i] = residue_sasa
        return total_sasa
    
    def b_factor(self, al_type):
        fixed_struct, moving_struct, key_name = self.load_structures()
        len_prot = [atom for residue in moving_struct.get_residues() for atom in residue]
        bfactor = 0
        for model in moving_struct:
            for chain in model:
                for residue in chain:
                    for atom in residue:
                        bfactor += atom.get_bfactor()
        b_factor = bfactor / len(len_prot)
        if al_type == 'esmfold':
            b_factor = b_factor * 100
        return b_factor

    def extract_sequence_from_pdb(self):
        fixed_struct, moving_struct, key_name = self.load_structures()
        model = moving_struct[0]
        ppb = Polypeptide.PPBuilder()
        for pp in ppb.build_peptides(model):
            return str(pp.get_sequence())
        
    def calculate_average_hydrophobicity(self, sequence):
        protein_analysis = ProteinAnalysis(sequence)
        hydrophobicity = protein_analysis.protein_scale(hw, window=7, edge=1.0)
        if len(hydrophobicity) == 0:
            return 'N/A'
        average_hydrophobicity = sum(hydrophobicity) / len(hydrophobicity)
        return average_hydrophobicity
    
    def calculate_total_hydrophobicity(self, sequence):
        total_hydrophobicity = 0
        for aa in sequence:
            if aa in kd:
                total_hydrophobicity += kd[aa]
        return total_hydrophobicity / len(sequence)

In [2]:
input_dir = '/home/max/stayahead/analysis/datasets/tmp/omicron_variants/esm/ba1_100'
ref_path = '/home/max/stayahead/analysis/datasets/references/esm/pdb/SARS-CoV-ACE2-ESM.pdb'
output_dir = '/home/max/stayahead/analysis/datasets/tmp/omicron_variants/esm'
results = []
for sub_dir, dirs, files in os.walk(input_dir):
    for file in files:
        pdb_path = os.path.join(input_dir, file)
        key_name = file.split('.')[0]
        metrics = ComputeMetrics(pdb_path, ref_path)
        rmsd = metrics.rmsd()
        tm_score = metrics.tm_score()
        sasa = metrics.sasa()
        plddt = metrics.b_factor('alpha')
        sequence = metrics.extract_sequence_from_pdb()
        avg_hydrophobicity = metrics.calculate_average_hydrophobicity(sequence)
        results.append([key_name, rmsd, tm_score, sasa, avg_hydrophobicity, plddt])
output_path = os.path.join(output_dir, 'ESM_BA1_100.csv')
with open(output_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['seq_id', 'rmsd', 'tm_score', 'sasa', 'avg_hydro', 'plddt'])
    writer.writerows(results)

FileNotFoundError: [Errno 2] No such file or directory: '/home/max/stayahead/analysis/datasets/tmp/omicron_variants/esm/ESM_BA1_100.csv'

In [6]:
pdb_path = '/Users/maxvandenboom/stayahead/analysis/datasets/references/alphafold/pdb/SARS-CoV-ACE2-AF_1.pdb'
af_path = '/Users/maxvandenboom/stayahead/analysis/datasets/references/alphafold/pdb/SARS-CoV-ACE2-AF_2.pdb'
esm_path = '/Users/maxvandenboom/stayahead/analysis/datasets/references/esm/pdb/SARS-CoV-ACE2-ESM.pdb'

metrics = ComputeMetrics(pdb_path, af_path)
rmsd = metrics.rmsd()
tm_score = metrics.tm_score()
print(rmsd)

0.8907628130241317
